In [65]:
import pandas as pd
from pandas import option_context
import numpy as np

import re
import string
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

### This is the main notebook for cleaning the app dataframe

In [66]:
df = pd.read_csv('04-data/scraped_app_data.csv')
df.info()

/Users/tawneykirkland/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46031 entries, 0 to 46030
Data columns (total 50 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   title                     46031 non-null  object 
 1   description               46025 non-null  object 
 2   descriptionHTML           46025 non-null  object 
 3   summary                   46009 non-null  object 
 4   summaryHTML               46009 non-null  object 
 5   installs                  46021 non-null  object 
 6   minInstalls               46021 non-null  float64
 7   score                     45932 non-null  float64
 8   ratings                   45877 non-null  float64
 9   reviews                   45932 non-null  float64
 10  histogram                 46021 non-null  object 
 11  price                     46021 non-null  float64
 12  free                      46021 non-null  object 
 13  currency                  46021 non-null  object 
 14  sale  

In [67]:
with open('non_english.pkl', 'rb') as f:
    non_eng_titles = pickle.load(f)

In [68]:
# Remove nonenglish apps discovered during topic modeling
df = df[~df['title'].isin(non_eng_titles)]

In [69]:
df = df.dropna(subset=['comments'])

In [70]:
df = df[df['ratings'] >= 1000]
df = df[df['reviews'] >= 100]

In [71]:
df = df[df['title'].duplicated() == False]

In [72]:
df = df[df['minInstalls'] == 1000000]

## Clean dataframe

In [73]:
df = df.drop(columns = ['descriptionHTML',
                       'summaryHTML',
                       'recentChangesHTML',
                       'developerEmail',
                       'developerWebsite',
                       'developerAddress',
                       'saleTime',
                       'originalPrice',
                       'saleText'])

In [74]:
# Drop duplicates
df = df.drop_duplicates(subset=['appId', 'title'], keep='last')

In [75]:
# Remove rows that do not have a value for ratings / and date released
df = df[df['ratings'].notna()]

df = df[df['released'].notna()]

In [76]:
# Remove additional non-english apps discovered during preprocess and analysis process

# By appID
df = df[df['appId'] != 'com.syc.librototal.El_Libro_Total']

# by developer name
developer_names = ['Boursorama','Dawat-e-Islami','Aplicaciones Cristianas','Nakagosoft, Bangladesh']

df = df[~df['developer'].isin(developer_names)]

In [77]:
# Year of release
df['year'] = pd.DatetimeIndex(df['released']).year

# Three apps released during 2021 - drop these given so few and not enough data
df = df[(df['year'] < 2021)]

In [78]:
# Convert the 'last updated' timestamp to datetime
from datetime import datetime

def get_date(time_stamp):
    date = datetime.fromtimestamp(time_stamp)
    
    return date

df['updated'] = df.updated.map(get_date)

In [79]:
# Year of last update
df['updated_year'] = pd.DatetimeIndex(df['updated']).year

In [80]:
# Clean description column
df['description'] = df['description'].astype(str)

def clean_text(text):
    '''Make text lowercase, remove punctuation, remove emojis, etc'''
    text = text.lower()
    text = re.sub(r"<[^>]*>", "", text)
    text = re.sub(r"(?:\@|https?\://)\S+", "", text)
    text = re.sub("[^a-zA-Z0-9 -]","",text)
    
    return text

df['description_clean'] = df.description.map(clean_text)

In [81]:
# Clean star histogram and separate into new columns
df['hist_clean'] = df['histogram'].apply(lambda x: x[1:-1].split(','))
df['hist_clean'] = df['hist_clean'].apply(lambda y: [int(x) for x in y])

In [82]:
df[['star_1','star_2','star_3','star_4','star_5']] = pd.DataFrame(df.hist_clean.tolist(), index= df.index)

In [83]:
# Reassign 'Educational' apps to 'Education' genre
df.loc[df['genre'] == 'Educational','genre'] = 'Education'

In [84]:
# Change data type to bool for analysis 
df['editorsChoice'] = df['editorsChoice'].astype(str)
df['editorsChoice'] = np.where(df['editorsChoice'] == 'True',1,0)

df['free'] = df['free'].astype(str)
df['free'] = np.where(df['free'] == 'True',1,0)

df['containsAds'] = df['containsAds'].astype(str)
df['containsAds'] = np.where(df['containsAds'] == 'True',1,0)

In [85]:
# Reassign a few apps
df.loc[(df['contentRating'] == 'Adults only 18+') |(df['contentRating'] == 'Unrated'),
      ['contentRating']] = 'Mature 17+'

df.groupby('contentRating')['contentRating'].count()

contentRating
Everyone        16523
Everyone 10+     1084
Mature 17+        896
Teen             3547
Name: contentRating, dtype: int64

In [86]:
df.groupby('updated_year')['updated_year'].count()

updated_year
2010       6
2011       9
2012      21
2013      91
2014     179
2015     297
2016     525
2017     827
2018    1447
2019    2704
2020    8746
2021    7198
Name: updated_year, dtype: int64

## Feature engineering

In [87]:
# Create aggregate view to find top developer by avg score per app
df_agg = df[['developer','minInstalls','score']]

df_agg['count'] = 1

df_agg = df_agg.groupby(['developer'])['minInstalls','count','score'].sum()
df_agg = pd.DataFrame(df_agg).reset_index()

df_agg['avg_installs'] = df_agg['minInstalls'] / df_agg['count']
df_agg['avg_score'] = df_agg['score'] / df_agg['count']

<ipython-input-87-0feed8a92e18>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_agg['count'] = 1
<ipython-input-87-0feed8a92e18>:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_agg = df_agg.groupby(['developer'])['minInstalls','count','score'].sum()


In [88]:
df_top = df_agg.sort_values('avg_score',ascending=False).head(1000)
unique_ids = df_top['developer'].tolist()

df['top_developer'] = np.where(df['developer'].isin(unique_ids),1,0)

In [89]:
df.groupby('top_developer').count()

,title,description,summary,installs,minInstalls,score,ratings,reviews,histogram,price,...,url,year,updated_year,description_clean,hist_clean,star_1,star_2,star_3,star_4,star_5
top_developer,,,,,,,,,,,,,,,,,,,,,
0,20745,20745,20744,20745,20745,20745,20745,20745,20745,20745,...,20745,20745,20745,20745,20745,20745,20745,20745,20745,20745
1,1305,1305,1305,1305,1305,1305,1305,1305,1305,1305,...,1305,1305,1305,1305,1305,1305,1305,1305,1305,1305


In [90]:
# Create a column for number of days since release
df['current_date'] = '2021-03-04'

df['days'] = (pd.DatetimeIndex(df['current_date']) - pd.DatetimeIndex(df['released'])).days

# Using minInstalls and days since release, create variable for average installs per day
df['installs_day'] = df['minInstalls']/ df['days']

In [91]:
# Create a column for number of days since last update
df['updated_days'] = (pd.DatetimeIndex(df['current_date']) - pd.DatetimeIndex(df['updated'])).days

In [92]:
# Does having a video to showcase the app make it more attractive? Create feature to capture video (yes / no)
df['has_video'] = np.where(df['video'].isna(),0,1)

df.groupby('has_video')['score'].mean()

has_video
0    4.036302
1    4.130232
Name: score, dtype: float64

In [93]:
df.shape

(22050, 56)

In [94]:
df.to_csv('04-data/preprocessed_app_data.csv',index=False)

In [95]:
df.sort_values('score',ascending=True).head(60)

,title,description,summary,installs,minInstalls,score,ratings,reviews,histogram,price,...,star_2,star_3,star_4,star_5,top_developer,current_date,days,installs_day,updated_days,has_video
6137,Ikariam Mobile,"As ruler of an island kingdom, you build up gr...","Construct, fight and research: let your island...","1,000,000+",1000000.0,1.324779,57642.0,18459.0,"[50837, 1859, 1149, 629, 3168]",0.0,...,1859,1149,629,3168,0,2021-03-04,2795,357.781753,1190,1
383,"TV Guide: Best Shows & Movies, Streaming & Liv...",TV Guide is the place to go for finding what t...,"Discover new series & films to stream, see cha...","1,000,000+",1000000.0,1.362267,43648.0,21819.0,"[36554, 3168, 1148, 768, 2010]",0.0,...,3168,1148,768,2010,0,2021-03-04,2859,349.772648,12,1
30373,Home Quarantine - Poland (Kwarantanna domowa),<b>Home Quarantine – Poland (Kwarantanna domow...,Polish Government’s app. It confirms location ...,"1,000,000+",1000000.0,1.402204,14224.0,10558.0,"[12188, 450, 450, 176, 960]",0.0,...,450,450,176,960,0,2021-03-04,350,2857.142857,7,0
4990,Style My Hair: Discover Your Next Look,Discover a new makeover experience and get ins...,"Get inspired with Style My Hair, L’Oréal Profe...","1,000,000+",1000000.0,1.492308,8452.0,3942.0,"[7042, 280, 190, 260, 680]",0.0,...,280,190,260,680,0,2021-03-04,2086,479.386385,50,1
12427,SBI Secure OTP,SBI Secure OTP is an OTP generation App for ve...,OTP generation App for SBI Internet Banking an...,"1,000,000+",1000000.0,1.519715,14745.0,7669.0,"[12354, 409, 181, 314, 1487]",0.0,...,409,181,314,1487,0,2021-03-04,1942,514.933059,1,1
27279,How Old am I?,It calculates how old you look like to the out...,It calculates how old you look like by analysi...,"1,000,000+",1000000.0,1.541436,5434.0,2418.0,"[4404, 270, 160, 50, 550]",0.0,...,270,160,50,550,0,2021-03-04,2298,435.161010,603,0
41857,TV9 Telugu,The Associated Broadcasting Company Pvt Ltd (A...,Tv9 Telugu Live,"1,000,000+",1000000.0,1.559055,14214.0,5944.0,"[11930, 119, 239, 359, 1567]",0.0,...,119,239,359,1567,0,2021-03-04,2558,390.930414,70,1
29967,Laser Pointer XXL - Simulator,"Start off with a simple red laser, then collec...",Have the power of a fake laser at your fingert...,"1,000,000+",1000000.0,1.589474,3758.0,2070.0,"[3067, 148, 29, 49, 465]",0.0,...,148,29,49,465,0,2021-03-04,1759,568.504832,203,0
29643,SWF Player,Play your flash files ( swf ) from your SD-car...,Play your flash files ( swf ) from your SD-car...,"1,000,000+",1000000.0,1.626476,16079.0,4835.0,"[12863, 509, 439, 389, 1879]",0.0,...,509,439,389,1879,0,2021-03-04,3352,298.329356,2442,0
33081,Make My Head Bald - Prank Photo Editor,Become bald instantly with the head change pho...,Hair Remover + Bald Head Camera + Face Change ...,"1,000,000+",1000000.0,1.628866,1942.0,884.0,"[1572, 50, 40, 30, 250]",0.0,...,50,40,30,250,0,2021-03-04,1273,785.545954,783,0


In [96]:
icons = df.head(5)
icons.columns

Index(['title', 'description', 'summary', 'installs', 'minInstalls', 'score',
       'ratings', 'reviews', 'histogram', 'price', 'free', 'currency', 'sale',
       'offersIAP', 'inAppProductPrice', 'size', 'androidVersion',
       'androidVersionText', 'developer', 'developerId', 'privacyPolicy',
       'developerInternalID', 'genre', 'genreId', 'icon', 'headerImage',
       'screenshots', 'video', 'videoImage', 'contentRating',
       'contentRatingDescription', 'adSupported', 'containsAds', 'released',
       'updated', 'version', 'recentChanges', 'comments', 'editorsChoice',
       'appId', 'url', 'year', 'updated_year', 'description_clean',
       'hist_clean', 'star_1', 'star_2', 'star_3', 'star_4', 'star_5',
       'top_developer', 'current_date', 'days', 'installs_day', 'updated_days',
       'has_video'],
      dtype='object')